# Installating Octave on Jupyter-Portable (www.portabledevapps.net) via a Notebook

In [ ]:
import os
import sys
import io

In [ ]:
try:
    import urllib.request as urllib2  # Python 3
except:
    import urllib2  # Python 2
if 'amd64' in sys.version.lower():
    octave_7z="octave-5.1.0-w64.7z"
    octave_url="https://ftpmirror.gnu.org/octave/windows/octave-5.1.0-w64.7z"
    hashes="b13549ad2f69654e25d51c70aad9dfe0635007303d855550c7c6dc2f73af8035" # sha256
else:
    octave_7z="octave-5.1.0-w32.7z"
    octave_url="https://ftpmirror.gnu.org/octave/windows/octave-5.1.0-w32.7z"
    hashes="b7a8fb8c78c781285ab12f4dedbdc07fdc42c3a03da812ffe2eaa77d49872618" # sha256
    
octave_archive=os.environ["APPSDIR"]+"\\datafiles\\"+octave_7z
# We neet 7zip to extract the archive. If the Julia kernel is installed, it will be in apps\Julia-<Version>\libexec
p7zexe=os.environ["APPSDIR"]+"\\Julia-1.3.1\\libexec\\7z.exe"

In [ ]:
# start download
g = urllib2.urlopen(octave_url) 
with io.open(octave_archive, 'wb') as f:
    f.write(g.read())
g.close
g = None

In [ ]:
# checking the hashes
import hashlib
def give_hash(of_file, with_this):
    with io.open(octave_archive, 'rb') as f:
        return with_this(f.read()).hexdigest()  
print (" "*30+"SHA-256"+" "*30+"\n"+"-"*65)

print ("%s %s" % (give_hash(octave_archive, hashlib.sha256),octave_archive))
assert give_hash(octave_archive, hashlib.sha256) == hashes

In [ ]:
#extract the 7z archive
from subprocess import run
extr_path = os.environ["APPSDIR"]
run(p7zexe+' x '+ octave_archive + ' -o' + extr_path)
os.remove(octave_archive)

In [ ]:
#rename
tools = os.listdir(os.environ["APPSDIR"])
for i in range(len(tools)):
    if tools[i][0:7] == "octave-":
        os.rename(os.environ["APPSDIR"]+"\\"+tools[i],os.environ["APPSDIR"]+"\\"+"octave")    

In [ ]:
# Installing iOctave via pip
!start cmd /C pip install octave_kernel

In [ ]:
# add octave environment variables to the Jupyter Environment
inp_str = r"""
rem ******************
rem handle octave if included
rem ******************
if not exist "%APPSDIR%\octave" goto octave_bad
set OCTAVE_EXECUTABLE=%APPSDIR%\octave\mingw64\bin\octave-cli.exe
set OCTAVE_KERNEL_JSON=%APPSDIR%\share\jupyter\kernels\octave\kernel.json
:octave_bad
"""

if not ('amd64' in sys.version.lower()):
    inp_str = inp_str.replace('mingw64','mingw32')

# the env.bat has the "hidden" file attribute and is therefore not writable
os.system("attrib -h "+os.environ["APPSDIR"]+"\\scripts\\env.bat")

with open(os.environ["APPSDIR"]+"\\scripts\\env.bat", "a") as myfile:
    myfile.write(inp_str)
    
# restore the hidden file attribute of env.bat
os.system("attrib +h "+os.environ["APPSDIR"]+"\\scripts\\env.bat")